In [1]:
import urllib.parse
import pandas as pd
from ipywidgets import widgets
from IPython.display import clear_output, display

BASE_URL = "https://portal.gss.stonybrook.edu/echoepa/?query="

In [2]:
GET_DB_SIZE = """
SELECT pg_size_pretty(pg_database_size(current_database())) AS size
"""

GET_TABLES = """
  SELECT table_name
    FROM information_schema.tables
   WHERE table_schema = 'public'
     AND table_type = 'BASE TABLE'
     AND table_name ILIKE '%%rsei%%'
"""

GET_COLUMNS_TEMPLATE = """
  SELECT column_name
    FROM information_schema.columns
   WHERE table_schema = 'public'
     AND table_name = %s
ORDER BY ordinal_position
"""

BASIC_SELECT_TEMPLATE = """
  SELECT %s
    FROM %s
"""

In [3]:
def _get_data_location(query):
    """
    This function takes a query string and returns the data location URL.
    """
    encoded_query = urllib.parse.quote_plus(query)
    data_location = f"{BASE_URL}{encoded_query}&pg"
    return data_location

def run_query(query):
    print(query)
    data_location = _get_data_location(query)
    ds = pd.read_csv(data_location,encoding='iso-8859-1')
    return ds

def get_columns(table_name):
    """
    This function takes a table name and returns the columns in that table.
    """
    query = GET_COLUMNS_TEMPLATE % f"'{table_name}'"
    columns = run_query(query)
    return columns

In [4]:
PING = "SELECT 1"

_get_data_location(PING)

'https://portal.gss.stonybrook.edu/echoepa/?query=SELECT+1&pg'

In [5]:
run_query(GET_DB_SIZE)


SELECT pg_size_pretty(pg_database_size(current_database())) AS size



,size
0,126 GB


In [6]:
rsei_tables = run_query(GET_TABLES)
rsei_tables


  SELECT table_name
    FROM information_schema.tables
   WHERE table_schema = 'public'
     AND table_type = 'BASE TABLE'
     AND table_name ILIKE '%%rsei%%'



,table_name
0,releases_data_rsei_v2312
1,elements_data_rsei_v2312
2,offsite_data_rsei_v2312
3,chemical_data_rsei_v2312
4,category_data_rsei_v2312
5,facility_data_rsei_v2312
6,naics_data_rsei_v2312
7,mcl_data_rsei_v2312
8,media_data_rsei_v2312
9,submissions_data_rsei_v2312


In [7]:
# OUTPUT DATAFRAME FROM THE QUERY CONSTRUCTED IN THE WIDGET
# ==========================================================
df = None
# ==========================================================

In [8]:
# TABLE SELECTION WIDGET (Dropdown)
selected_table = widgets.Dropdown(
    options=rsei_tables['table_name'].tolist(),
    value=rsei_tables['table_name'].tolist()[0],
    description='Tables',
    disabled=False
)
table_selection_summary = widgets.HTML(
    value="Table: " + selected_table.value,
    layout=widgets.Layout(padding='10px 0px 0px 0px')
)

def on_change_table(change):
    selected_columns.options = get_columns(change['new'])['column_name'].tolist()
    selected_columns.value = selected_columns.options
    table_selection_summary.value = "Table: " + change['new']

selected_table.observe(on_change_table, names='value')

# COLUMN SELECTION WIDGET (Multi-select)
selected_columns = widgets.SelectMultiple(
    options=get_columns(selected_table.value)['column_name'].tolist(),
    value=get_columns(selected_table.value)['column_name'].tolist(),
    description='Columns',
    disabled=False,
    layout=widgets.Layout(width='50%', height='200px')
)
columns_selection_summary = widgets.HTML(
    value="Columns: " + ", ".join(selected_columns.value),
    layout=widgets.Layout(padding='10px 0px 0px 0px')
)

def on_change_columns(change):
    columns_selection_summary.value = "Columns: " + ", ".join(change['new'])

selected_columns.observe(on_change_columns, names='value')

# BUTTON TO FETCH THE RSEI DATA
fetch_data_button = widgets.Button(description="Fetch data", button_style='success')
df_output = widgets.Output()

def _escape_column_names(columns):
    """
    This function takes a list of column names and returns a list of escaped column names.
    """
    return [f'"{col}"' for col in columns]

def on_fetch_data_clicked(button):
    with df_output:
        global df
        clear_output()
        selected_columns_str = ", ".join(_escape_column_names(selected_columns.value))
        query = BASIC_SELECT_TEMPLATE % (selected_columns_str, selected_table.value)
        show_spinner()
        df = run_query(query)
        hide_spinner()
        if df.empty:
            print("No data found.")
        else:
            display(df)
            
fetch_data_button.on_click(on_fetch_data_clicked)


  SELECT column_name
    FROM information_schema.columns
   WHERE table_schema = 'public'
     AND table_name = 'releases_data_rsei_v2312'
ORDER BY ordinal_position


  SELECT column_name
    FROM information_schema.columns
   WHERE table_schema = 'public'
     AND table_name = 'releases_data_rsei_v2312'
ORDER BY ordinal_position



In [9]:
# Spinner HTML
spinner_html = """
<div class="loader" style="
  border: 4px solid #f3f3f3;
  border-top: 4px solid #3498db;
  border-radius: 50%;
  width: 24px;
  height: 24px;
  animation: spin 1s linear infinite;
">
<style>
@keyframes spin {
  0% { transform: rotate(0deg); }
  100% { transform: rotate(360deg); }
}
</style>
</div>
"""
spinner = widgets.HTML(spinner_html)
container = widgets.VBox([])

def show_spinner():
    if spinner not in container.children:
        container.children = [spinner]

def hide_spinner():
    container.children = []

In [10]:
selection_widgets = widgets.HBox(
    [selected_table, selected_columns], 
    layout=widgets.Layout(
        width='100%', 
        height='300px',
    )
)

display(widgets.VBox(
    [
        selection_widgets,
        table_selection_summary,
        columns_selection_summary,
        fetch_data_button,
        df_output
    ],
    layout=widgets.Layout(
        width='100%',
        height='auto',
        justify_content='space-around',
    )
), container)

VBox()

In [12]:
df

,ReleaseNumber,SubmissionNumber,PoundsReleased
0,9977861,2998525,103.33
1,9977862,2998525,103.33
2,9977863,2998525,103.33
3,9977864,2998522,2.67
4,9977865,2998520,266.67
...,...,...,...
7779853,9977856,2998530,257.14
7779854,9977857,2998530,257.14
7779855,9977858,2998530,257.14
7779856,9977859,2998530,257.14
